In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from shutil import rmtree
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import json



In [2]:
# run_dir1 = Path('./experiment-240823-231202/result/') # workstation test
# run_dir2 = Path('./experiment-270823-185523/result/') # test Slurm 
# run_dir3 = Path('./experiment-270823-190351/result/') # workstation test 2
run_dir4 = Path('/home/lizz_lab/cse12232433/project/colmena/multisite_/molecular-design/runs/saved_runs/xtb-N32-n1-e2f8b5-10Aug23-163733') # molecule design test
run_dir5 = Path('/home/lizz_lab/cse12232433/project/colmena/multisite_/finetuning-surrogates/runs/dft-md-23Aug02-140517-dbbb67') # finetune-surrogates test

In [3]:
### Load the data
task_type = ['inference-results', 'training-results', 'simulation-results']
task_color = {'run_simulation': 'blue', 'retrain_mpnn': 'red', 'evaluate_mpnn': 'yellow'}
def load_results(path: Path, task_type) -> pd.DataFrame:
    """Load the results from a run.
    
    Args:
        path: Path to the run directory.
    Returns:
        Dataframe of all results
    """
    
    # Loop over inference, training, and simulation results
    output = []
    for task in task_type:
        # Load the results
        data = path / f'{task}.json'
        assert data.is_file(), f'Could not find {data}'
        results = pd.read_json(data, lines=True)
        output.append(results)
    output = pd.concat(output)
    return output


In [4]:
def time_line_graph(results: pd.DataFrame, task_color):
    #### time line of task
    # rows = len(results)
    fig = plt.figure(figsize=(100,100))
    task_timeline = results
    task_timeline.sort_values('time_compute_started', inplace=True, ignore_index=True)
    start_time = task_timeline['time_compute_started'].loc[0]
# type_color = {'inference': 'red', 'train': 'green', 'simulation': 'blue'}
    # type_color = {'run_simulation': 'blue', 'retrain_mpnn': 'red', 'evaluate_mpnn': 'yellow'}
    for index, row in task_timeline.iterrows():
        print(f"{row['method']}: {row['time_compute_started']}: {row['time_running']}")
        method = str(row['method'])+str(index)
        time = row['time_running']
        plt.barh(index, time, left=row['time_compute_started'] - start_time, color=task_color[row['method']])
        plt.text(row['time_compute_started'] - start_time, index, method, ha='left', va='center')
    
    plt.xlabel('Time')
    plt.ylabel('Task')
# 显示图形
    plt.tight_layout()
    plt.show()

In [5]:
results = load_results(run_dir4, task_type=task_type)
time_line_graph(results=results, task_color=task_color)


retrain_mpnn: 1691685466.621303: 81.36718778312206
retrain_mpnn: 1691685466.764188: 79.79135167808272
retrain_mpnn: 1691685466.82452: 70.19868533313274
retrain_mpnn: 1691685466.980746: 57.60286928713322
retrain_mpnn: 1691685524.646161: 80.46453482308425
retrain_mpnn: 1691685537.050857: 69.50932551710866
evaluate_mpnn: 1691685546.70278: 9.25565446098335
evaluate_mpnn: 1691685548.118877: 8.584165453910828
evaluate_mpnn: 1691685556.996317: 8.352525118039921
evaluate_mpnn: 1691685557.764238: 8.416042685043067
evaluate_mpnn: 1691685566.190448: 8.35980074503459
evaluate_mpnn: 1691685566.981134: 8.230650760931894
evaluate_mpnn: 1691685575.519003: 8.316578881815076
evaluate_mpnn: 1691685576.166237: 8.349251681007445
evaluate_mpnn: 1691685584.57436: 8.410889795981348
evaluate_mpnn: 1691685585.244443: 8.22298163198866
evaluate_mpnn: 1691685593.826311: 8.27534823003225
evaluate_mpnn: 1691685594.292953: 8.275484900921583
evaluate_mpnn: 1691685603.072807: 8.159506679978222
evaluate_mpnn: 1691685603

In [6]:
task_type = ['sampling-results', 'training-results', 'simulation-results']
task_color = {'run_calculator': 'blue', 'train': 'red', 'run_sampling': 'yellow'}
results = load_results(run_dir5, task_type=task_type)
time_line_graph(results=results, task_color=task_color)

train: 1690985130.85057: 154.22281202604063
train: 1690985130.875632: 153.53094223095104
train: 1690985130.943893: 154.11106904502958
train: 1690985130.964922: 155.23044948210008
run_sampling: 1690985287.417861: 385.4519893710967
run_sampling: 1690985287.419694: 433.8305668849498
run_sampling: 1690985287.419855: 431.46415576920845
run_sampling: 1690985287.420263: 423.98021684307605
run_sampling: 1690985680.364331: 274.32280164095573
run_sampling: 1690985714.405366: 356.2345217319671
run_sampling: 1690985722.138795: 34.339083408005536
run_sampling: 1690985723.886284: 407.0815017968416
run_sampling: 1690985761.10821: 429.8980247809086
run_sampling: 1690985961.668759: 36.15150529285893
run_sampling: 1690986001.291071: 73.2390107519459
run_sampling: 1690986075.524675: 398.57540010195225
run_sampling: 1690986077.383844: 449.08001794992015
run_sampling: 1690986134.606776: 405.0981982150115
run_calculator: 1690986192.030708: 0.08414309308864101
run_calculator: 1690986192.645359: 0.05017932294

In [13]:
### task on nodes and gpu
row_count = len(results)
print(row_count)

308
